In [13]:
from bs4 import BeautifulSoup as BS
from urllib.request import urlopen

In [14]:
html = urlopen("https://www.pythonscraping.com/pages/page1.html")

In [15]:
bs = BS(html.read(), 'html.parser')
print(bs.head)

<head>
<title>A Useful Page</title>
</head>


In [16]:
from urllib.error import HTTPError

In [20]:
def getTtitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    
    try:
        bs = BS(html.read(), 'html.parser')
        title = bs.body.h1
        
    except AttributeError as e:
        return None;
    
    return title

In [21]:
title = getTtitle('https://www.pythonscraping.com/pages/page1.html')
if title == None:
    print("Title is None")
else:
    print(title)

<h1>An Interesting Title</h1>


In [22]:
html = urlopen('https://www.pythonscraping.com/pages/warandpeace.html')
bs = BS(html, 'html.parser')

In [24]:
nameList = bs.findAll('span', {'class' : 'green'})
for name in nameList:
    print(name.get_text(),end=" ")

Anna
Pavlovna Scherer Empress Marya
Fedorovna Prince Vasili Kuragin Anna Pavlovna St. Petersburg the prince Anna Pavlovna Anna Pavlovna the prince the prince the prince Prince Vasili Anna Pavlovna Anna Pavlovna the prince Wintzingerode King of Prussia le Vicomte de Mortemart Montmorencys Rohans Abbe Morio the Emperor the prince Prince Vasili Dowager Empress Marya Fedorovna the baron Anna Pavlovna the Empress the Empress Anna Pavlovna's Her Majesty Baron
Funke The prince Anna
Pavlovna the Empress The prince Anatole the prince The prince Anna
Pavlovna Anna Pavlovna 

In [29]:
html = urlopen('https://www.pythonscraping.com/pages/page3.html')
bs = BS(html, 'html.parser')

In [30]:
print(bs.h1)

<h1>Totally Normal Gifts</h1>


In [33]:
for child in bs.find('table', {'id': 'giftList'}).children:
    print(child)



<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>


In [35]:
for child in bs.find('table', {'id': 'giftList'}).tr.next_siblings:
    print(child)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [36]:
print(bs.find('img', {'src': '../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())


$15.00

